In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,575 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Err:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-2").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
|R2J2KMDL10UMSH|          5|            0|          0|   N|                Y|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|                Y|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|                Y|
|R1FXZ69C01JNQM|          5|            0|          0|   N|                Y|
| RY36LB4OW0FFS|          5|            0|          0|   N|     

In [ ]:
#Review typeData
vine_df

DataFrame[review_id: string, star_rating: string, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]

In [ ]:
#Filter the data and create a new DataFrame to retrieve all the rows where the total_votes count is equal to or greater than 20.
vine_df2 = df.filter(df.total_votes >= 20).show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

In [ ]:
#create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
vine_df3 =  vine_df2.filter((vine_df2.helpful_votes / vine_df2.total_votes) >= 0.5).show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

In [ ]:
#Load in a sql function to use columns
from pyspark.sql.functions import col
#retrieves all the rows where a review was written as part of the Vine program (paid)
vine_df4 = vine_df3.filter(col("vine")=='Y')
vine_df4.show() 

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51635935|R15PXA9XU9ZKSU|B00TDNZV4A|     280424829|Sterling Silver C...|         Jewelry|          4|           29|         29|   Y|                N|So classy and gor...|You can wear thes...| 2015-05-09|
|         US|   52833354|R2LJ4QYQ4PYDEH|B00TDNZUYG|     280424829|Sterling Silver C...|         Jewelry|          5|    

In [ ]:
#retrieves all the rows where a review was written as part of the Vine program (paid)
vine_df5 = vine_df3.filter(col("vine")=='N').show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

In [ ]:
#Total number of reviews vine program paid
Total_Reviews_paid =  vine_df4.count()
Total_Reviews_paid 

21

In [ ]:
#Total number of reviews vine program unpaid
Total_Reviews_unpaid =  vine_df5.count()
Total_Reviews_unpaid

7689

In [ ]:
#Total number of 5 star-Review vine program paid
Total_StarReview_paid = vine_df4.where(vine_df4.star_rating ==5).count()
Total_StarReview_paid

11

In [ ]:
#Total number of 5 star-Review vine program unpaid
Total_StarReview_unpaid= vine_df5.where(vine_df5.star_rating ==5).count()
Total_StarReview_unpaid

4444

In [ ]:
#Percentage of 5 star-Review vine program paid
percentageSR_paid = (Total_StarReview_paid / Total_Reviews_paid) *100
percentageSR_paid

52.38095238095239

In [ ]:
#Percentage of 5 star-Review vine program unpaid
percentageSR_unpaid = (Total_StarReview_unpaid / Total_Reviews_unpaid ) * 100
percentageSR_unpaid

57.79685264663805